# Scraper de Red Academica

La página redacademica.edu.co contiene un catálogo de colegios en Bogotá, con información también del código DANE. Los nombres de los colegios cuadran bastante bien con los encontrados en el archivo de base de colegios.

Este Notebook desarrollará el scraper de esta página web independientemente de los colegios que se encuentran en el archivo de búsqueda. Posteriormente se realizará el emparejamiento de la información.

In [1]:
from urllib.parse import unquote
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import time
import os

real_academia_url = 'https://www.redacademica.edu.co/colegios?name=&field_localidad_target_id=All&page={idx}'
school_base_url = 'https://www.redacademica.edu.co'

In [2]:
def get_school_batch_urls(idx):
  """
  Permite obtener las URLs de los colegios de la página:

  Params
  --------

    idx: (int):
      Un enetero que indica que página de colegios está revisando. (De 1 a 14)

  Returns
  --------
    batch_urls (list):
      Un diccionario que contiene la URL de cada colegio de la página
  """

  # Request de Extracción
  response = requests.get(real_academia_url.format(**{'idx': idx}))
  soup = BeautifulSoup(response.text, 'html.parser')
  school_elements = soup.find_all(class_='card-school__name')
  anchor_elements = soup.find_all('a', class_='card-school__name')

  # Extracción de hrefs de colegios
  school_links = {}
  for anchor in anchor_elements:
      school_name = anchor.text.strip()
      href = anchor['href']
      school_links[school_name] = school_base_url + href

  return school_links

In [3]:
def request_school_info(school_url):
  """
  Permite extraer la información de los colegios en la Information Box.

  Params
  --------
    school_url (str):
      La URL del colegio que se va a extraer.

  Returns
  --------
    school_info (dict):
      Un diccionario que contiene toda la información que pudo extraerse del colegio.

  """
  response = requests.get(school_url)
  soup = BeautifulSoup(response.text, 'html.parser')

  # Extracción de Information Box
  school_info_list = soup.find(class_='school-info__list')



  school_info = {}
  # Procesamiento del body de la página y extracción de información
  if school_info_list:
      for item in school_info_list.find_all('li', class_='school-info__list-item'):

          label = item.find('h4').get_text().strip().lower()

          content_list = []
          for div in item.find_all('div', class_='field__item'):
              content_list.append(div.get_text().strip())

          if not content_list:
              p_tag = item.find('p')
              if p_tag and not p_tag.find('div'):
                  content_list.append(p_tag.get_text().strip())
              a_tag = item.find('a')
              if a_tag:
                  href = a_tag['href']
                  text = a_tag.get_text(strip=True)
                  if 'mailto:' in href:
                      content_list.append(unquote(href.split(':', 1)[-1]))
                  elif 'tel:' in href:
                      content_list.append(text)
                  else:
                      content_list.append(unquote(a_tag.get_text(strip=True)))


          content = ', '.join(content_list) if content_list else None
          school_info[label] = content

  return school_info

# Utilización

En la sección anterior se desarrolló el código para la extracción de información de los colegios. Es necesario iterar sobre las 15 páginas de listados de colegios para extraer la informción de cada uno de manera individual.

Los resultados de la extracción se almacenarán por índice en el archivo `red_academica_information.csv` para su posterior consulta.

La extracción se hará con un rezago de 1 segundo para evitar bloqueos.

In [4]:
output_file = 'red_academica_information.csv'

In [5]:
for idx in tqdm(range(15)):
  school_urls = get_school_batch_urls(idx)
  complete_school_extraction = []
  for school_name, school_url in school_urls.items():
    school_info = request_school_info(school_url)
    school_info['nombre'] = school_name
    school_info['idx'] = idx
    complete_school_extraction.append(school_info)
    time.sleep(1)

  # Estructuración en Data Frame
  df = pd.DataFrame(complete_school_extraction)
  df.drop_duplicates(inplace=True)
  # Append si existe, y creación en primera iteración
  if os.path.isfile(output_file):
    df.to_csv(output_file, mode='a', header=False, index=False)
  else:
    df.to_csv(output_file, mode='w', header=True, index=False)

100%|██████████| 15/15 [06:04<00:00, 24.32s/it]
